# Проект для «Викишоп» с Bert

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [ ]:
import numpy as np
import pandas as pd
import torch
import warnings

from tqdm import notebook
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score


In [ ]:
warnings.filterwarnings('ignore',category=Warning)

## Подготовка

Тут попытаемся прочитать файл, он у меня локально

In [ ]:
try:
    #df_text = pd.read_csv('https://code.s3.yandex.net//toxic_comments.csv')
    df_text = pd.read_csv('C:/ya/toxic_comments.csv')
except:
    df_text = pd.read_csv('/datasets/toxic_comments.csv')

Посмотрим на данные

In [ ]:
df_text

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [ ]:
df_text['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

У нас много лишних символов, но так как будем использовать Bert, то можно их не удалять. Модель должна справиться

## Подготовка

Загрузка предобученной модели/токенизатора 

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#Ограничим в 500 строк, иначе зависает
features = df_text['text'][:500]
target = df_text['toxic'][:500]

Посмотрим размер

In [ ]:
target.value_counts()

0    450
1     50
Name: toxic, dtype: int64

Преобразуем каждое предложение в список идентификаторов (токенов). 
Возьмем 512, так как этор максимальная длинна, которую может принять Bert

In [ ]:
notebook.tqdm.pandas()
tokenized = features.progress_apply((lambda x: tokenizer.encode(x[:512], add_special_tokens=True)))

Применим метод padding, чтобы после токенизации длины исходных текстов в корпусе были равными, при таком условии будет работать модель BERT. 

In [ ]:
padded = np.array([i + [0]*(512 - len(i)) for i in tokenized.values])

Создадим маску для важных токенов

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)

padded.shape,attention_mask.shape

((500, 512), (500, 512))

Преобразуем тексты в векторных формат

In [ ]:
batch_size = 10
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

Соберём все эмбеддинги в матрицу признаков вызовов функции concatenate

In [ ]:
features_embeddings = np.concatenate(embeddings)
features_embeddings.shape

(500, 768)

Разделим выборку

In [ ]:
features_embeddings_train,features_embeddings_test,target_embeddings_train,target_embeddings_test = train_test_split(features_embeddings,target,test_size=0.2,random_state=0,stratify=target)

## Обучение

Подберем лучшие парметры

In [ ]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(features_embeddings_train, target_embeddings_train)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scrores:  0.9450000000000001


In [ ]:
lr= LogisticRegression(C = 5.263252631578947)
lr.fit(features_embeddings_train,target_embeddings_train)

LogisticRegression(C=5.263252631578947)

In [ ]:
predictions = lr.predict(features_embeddings_test)

In [ ]:
print('Метрика f1 модели DistilBertModel {:.2f}'.format(f1_score(target_embeddings_test, predictions)))

Метрика f1 модели DistilBertModel 0.75


## Выводы

У нас получилось создать инструмент, который поможет магазину выяснять в автоматическом режиме, что надо отправлять на модерацию, а что нет.

Метрика f1 у нас получилась равно 0.75. При этом было использовано всего 500 строк из Датасета, скорее всего с более мощной машиной удастся достичь лучших результатов